### Fitting Naive Bayes to the Training set

posterior = prior occurrances * liklihood / evidence
The "gaussian" and "naive" come from two assumptions present in this likelihood:
1. we assume each feature is uncorrelated from each other. That is, meannfreq is                  independent of sd or median etc.. This is obviously not true, and is a "naive"                assumption.
2. we assume have that the value of the features (e.g. meannfreq of women) are normally          (gaussian) distributed. This means that p(meannfreq∣female) is calculated by inputing          the required parameters into the probability density function of the normal                    distribution.


In [1]:
from sklearn.naive_bayes import GaussianNB
import pandas as pd 
import numpy as np  
import random
from sklearn.model_selection import train_test_split  
from collections import defaultdict

In [8]:
voices = voice_data.values[:, :-1] 
labels = voice_data.values[:, -1:]
# male -> 1
# female -> 0
from sklearn.preprocessing import LabelEncoder
gender_encoder = LabelEncoder()
labels = gender_encoder.fit_transform(labels)

# randomly shuffle data
voices_tmp = []  
lables_tmp = []  
index_shuf = range(len(voices)) 
random.shuffle(index_shuf) 
for i in index_shuf:  
    voices_tmp.append(voices[i])  
    lables_tmp.append(labels[i])  
voices = np.array(voices_tmp)  
labels = np.array(lables_tmp) 
train_x, test_x, train_y, test_y = train_test_split(voices, labels, test_size=0.2) 

/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [2]:
# For both the classes 0 and 1, and for each of the 20 features, we have thus saved the mean and the standard in summary. 
# For example, for the class 0 (female), and feature 1 (ie, sd), mean and the standard for this attribute are 0.060910268026077803 and 0.00034668419803097784

def getSummary(data,n_cls):
    data_means = data.groupby('label').mean()
    data_variance = data.groupby('label').var()
    
    cnames = np.array(data.columns.tolist())[:-1] 
    classes = np.unique(labels)
    classnames = np.array(['female','male'])
    summary = {}

    for cls in classes:
        #initializing the dictionary
        summary[cls] = defaultdict(list)
    for cls in classes:
        for j in range(0, n_cls):
            summary[cls][j] += list(np.array([data_means[cnames[j]][data_means.index == classnames[cls]].values[0],
                                          data_variance[cnames[j]][data_variance.index == classnames[cls]].values[0]]))
            
    print summary   
    return summary

In [5]:
# Group the data by gender and calculate the means of each feature
data_means = voice_data.groupby('label').mean()
# View the values
data_means

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,mode,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx
label,,,,,,,,,,,,,,,,,,,,
female,0.182094,0.060910,0.191728,0.147024,0.225485,0.078462,4.064059,54.002627,0.901429,0.412886,0.143478,0.182094,0.181745,0.057006,0.254705,0.388785,0.079212,1.004110,0.924898,0.276122
male,0.159523,0.069073,0.158513,0.105891,0.217190,0.111299,4.263501,68.912400,0.926243,0.522905,0.107962,0.159523,0.118485,0.039022,0.209773,0.328626,0.047011,1.082099,1.035088,0.293448


In [3]:
# Create a function to calculate the probability density of each of the terms of the likelihood 
# We use mean and standard deviation of input values (x) for each class to summarize the distribution
# Probabilities of new x values are calculated using the Gaussian Probability Density Function (PDF).

def p_x_given_y(x, mean_y, variance_y):
    # Input the arguments into a probability density function
    p = 1/(np.sqrt(2*np.pi*variance_y)) * np.exp((-(x-mean_y)**2)/(2*variance_y))
    return p

# We calculate the probability that the value of the random variable will be between x - 0.01 and x + 0.01


In [6]:
# Group the data by gender and calculate the variance of each feature
data_variance = voice_data.groupby('label').var()
# View the values
data_variance

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,mode,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx
label,,,,,,,,,,,,,,,,,,,,
female,0.001231,0.000347,0.001777,0.003629,0.000636,0.002290,19.981937,19590.277614,0.001270,0.034562,0.009534,0.001231,0.001053,0.002982,0.000481,0.061784,0.031561,0.497947,0.472875,0.010881
male,0.000928,0.000154,0.001623,0.001889,0.000803,0.000976,30.680457,31771.580085,0.000968,0.031902,0.007584,0.000928,0.000701,0.001223,0.003038,0.047032,0.005909,0.804810,0.792567,0.012728


In [7]:
classnames = np.array(['female','male'])
classnames[0]

'female'

In [4]:
p_x_given_y(2, 2, 4)

0.19947114020071635

In [5]:
def calculateClassProbabilities(summaries, inputVector):
    # p(x|y) (e.g. p(meannfreq∣female))
    probabilities = {}
    for classValue, classSummaries in summaries.iteritems():
        if (classValue == 0):
            probabilities[classValue] = P_female
        
        else:
            probabilities[classValue] = P_male
        for i in range(len(classSummaries)):
            mean, var = classSummaries[i]
            x = inputVector[i]
            probabilities[classValue] *= p_x_given_y(x, mean, var)
    return probabilities

In [6]:
summary = getSummary(voice_data,train_x.shape[1])

NameError: name 'voice_data' is not defined

In [ ]:
calculateClassProbabilities(summary, test_x[10])
# PDF can be greater than 1, but it can't be greater than 1 for a large interval
# because of the non-negativity and integral constraints.

In [ ]:
# Make a Prediction
import operator
def predict(summaries, inputVector):
    probabilities = calculateClassProbabilities(summaries, inputVector)
    #print probabilities
    probabilities = sorted(probabilities.iteritems(), key=operator.itemgetter(1), reverse=True)
    return probabilities[0][0]

In [ ]:
print(test_x[10])
print('Our prediction for test_x[10] is %s'  % (predict(summary, test_x[10])))

In [ ]:
from sklearn.metrics import classification_report

def GaussianNB_Classifier(test_x,test_y):
    results = []
    for i in range(len(test_x)):
        test_single = test_x[i]
        res = predict(summary, test_single)
        results.append(res)
    # We evaluate the accuracy of the model by calculating a ratio of the total correct predictions out of all predictions made (the classification accuracy)
    correct = 0
    wrong = 0
    for j in range(len(test_y)):
        if(results[j] == test_y[j]):
            correct += 1
        else:
            wrong += 1
 
    print('Correct rate is %s'  % (float(correct)/(correct+wrong)))
    print(classification_report(test_y,results))



In [ ]:
print("Use classification reports to evaluate how well Gaussian Naive Bayes performed with all features \n")
print("in-sample accuracy in Gaussian Naive Bayes:")
GaussianNB_Classifier(train_x,train_y)

In [ ]:
print("out-of-sample accuracy in Gaussian Naive Bayes:")
GaussianNB_Classifier(test_x,test_y)

In [ ]:
# We are wondering if there is a big difference in the results 
# when used Sklearn package vs. our own algorithm
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

nb = GaussianNB()
nb.fit(train_x, train_y)
prediction = nb.predict(test_x)

print('NB result: ', accuracy_score(test_y, prediction))

In [ ]:
# Is it enough of these two features to make predictions? 
## Random Forest with 2 features ('meanfun', 'IQR')

voice_new_data = voice_data[['IQR','meanfun','label']]
train_new_x = train_x[:,[5,12]]
test_new_x = test_x[:,[5,12]]
summary = getSummary(voice_new_data, train_new_x.shape[1])

In [ ]:
print("Use classification reports to evaluate how well Gaussian Naive Bayes performed with only 2 features \n")
print("In-sample accuracy in Gaussian Naive Bayes:")
GaussianNB_Classifier(train_new_x,train_y)

In [ ]:
print("Out-of-sample accuracy in Gaussian Naive Bayes:")
GaussianNB_Classifier(test_new_x,test_y)

We did observe some improvement in performance when we implement Gaussian Naive Bayes with 2 features.